# In this notebook we display the performance of different versions of damped Newton with preconditioning.

In [ ]:
from __future__ import division
import os
import numpy as np
import time
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
np.random.seed(1234)
%matplotlib inline
%load_ext autoreload
%autoreload 

In [ ]:
relative_path_to_new_folder = "../Images"
os.makedirs(relative_path_to_new_folder, exist_ok = True)
if not os.path.isdir('../Images/DampedNewtonPreconditioningAllversions_images'):
    os.makedirs('../Images/DampedNewtonPreconditioningAllversions_images')

In [ ]:
"""To compute distance matrix"""
def distmat( x, y ):
    return np.sum( x**2, 0 )[:,None] + np.sum( y**2, 0 )[None,:] - 2 * x.transpose().dot( y )

"""To Normalise a vector"""
normalize = lambda a: a/np.sum( a )

"""To Compute P"""
def GetP( u, K, v ):
    return u[:,None] * K * v[None,:]

def plotp( x, col, plt, scale = 200, edgecolors = "k" ):
  return plt.scatter( x[0,:], x[1,:], s = scale, edgecolors = edgecolors,  c = col, cmap = 'plasma', linewidths = 2 )

In [ ]:
N = [ 400, 500 ]

In [ ]:
x = np.random.rand( 2, N[0] ) - 0.5
theta = 2 * np.pi * np.random.rand( 1, N[1] )
r = 0.8 + .2 * np.random.rand( 1, N[1] )
y = np.vstack( ( r * np.cos( theta ), r * np.sin( theta ) ) )

In [ ]:
import computational_OT

# 0. Sinkhorn vs damped Newton

# Entropy regularized formulation

The primal entropy regularized formulation of OT is given by:
$$
OT_{\varepsilon}(\alpha,\beta) = min_{\pi \in \mathcal{U}(\alpha,\beta)} \langle C,\pi \rangle +\varepsilon KL(\pi\|\alpha \otimes \beta)\ ,
$$
where
$\ 
KL(\pi\|\alpha \otimes \beta) 
\ $ is the KL-divergence and $\ \mathcal{U}(\alpha,\beta)=\{\pi: \pi\mathcal{1}=\alpha, \pi^{T}\mathcal{1}=\beta\}$.

# 0.1. Sinkhorn

The optimal coupling $\pi^{*}$ has the following form :
$$
\pi^{*} = \alpha \odot diag(u)K diag(v)\odot \beta
$$
and we know that $\pi^{*}\mathbb{1}=\alpha$ and $(\pi^{*})^{T}\mathbb{1}=\beta$.
###
Therefore, Sinkhorn updates is given by the following alternative projections
$$
u^{t+1}  \leftarrow \frac{1}{K(v^{t}\odot \beta)}\ , \
v^{t+1}  \leftarrow \frac{1}{K^{T}(u^{t+1}\odot \alpha)}\ , 
$$
where 
$K = e^{-\frac{C}{\varepsilon}}\in M_{n\times m}(\mathbb{R}),\ \alpha \in \mathbb{R}^{n},\ \beta \in \mathbb{R}^{m}\ ,\ u\in \mathbb{R}^{n},\ v\in \mathbb{R}^{m}\ and \ (u^{0},v^{0})=(u,v)\ .$


In [ ]:
# Sinkhorn
print("Sinkhorn... ")
print("Doing for (",N[0], N[1],").")
SinkhornP = []
results_Sinkhorn = []
times_Sinkhorn = []
epsilons = [ 1.0, 0.5, 0.1, 0.05, 0.03 ]
#Cost matrix
C = distmat( x, y )
# a and b
a = normalize( np.ones( N[0] ) )
b = normalize( np.ones( N[1] ) )
for eps in epsilons:
  #Kernel
  K = np.exp( - C/eps )
  print( " |- Iterating")
  #Inflating
  u = a
  v = b
  start = time.time()
  Optimizer = computational_OT.sinkhorn(  K,
                                          a,
                                          b,
                                          u,
                                          v,
                                          eps )
  out = Optimizer._update()
  results_Sinkhorn.append( out )
  end = time.time()
  times_Sinkhorn.append( end - start )
  print( " |- Computing P" )
  print( "" )
  u_opt = np.exp( out['potential_f']/eps )
  K = np.exp( - C/eps )
  v_opt =  np.exp( out['potential_g']/eps )
  P_opt = GetP( u_opt, K, v_opt )
  SinkhornP.append( P_opt )
# end for

### Error plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.subplot( 2, 1, 1 ),
plt.title( "$||P1 -a||_1+||P1 -b||_1$" )
for i in range( len(results_Sinkhorn) ):
  error = np.asarray( results_Sinkhorn[i]['error_a'] ) + np.asarray( results_Sinkhorn[i]['error_b'] )
  plt.plot( error,label = 'Sinkhorn for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
# end for
plt.yscale( 'log' )
plt.legend()
plt.savefig( "../Images/DampedNewtonPreconditioningAllversions_images/ConvergenceSinkhornvaryingepsilon.pdf", format = 'pdf' )
plt.show()

# 0.2. Damped Newton

## Damped Newton algorithm:
The dual formulation of OT is given by
$$
OT_{\varepsilon} = \max_{f\in \mathbb{R}^{n}, g\in\mathbb{R}^{m}} \langle f, \alpha \rangle + \langle g, \beta \rangle - \varepsilon\left(\langle\alpha \otimes \beta, e^{\frac{f}{\varepsilon}}\odot K \odot e^{\frac{g}{\varepsilon}}  \rangle-1\right)\ ,
$$
where
$$
\alpha \in \mathcal{M}_{1}(\mathcal{X}),\ \beta \in \mathcal{M}_{1}(\mathcal{Y}),\ \varepsilon>0,\ f\in\mathbb{R}^{n},\ g\in \mathbb{R}^{m}\ .
$$

The Hessian is given by 
$\nabla^{2}Q_{\alpha, \beta,\varepsilon}(f,g)=\frac{-1}{\varepsilon}
\begin{pmatrix}
\Delta(\alpha) && \pi_{\varepsilon}\\
\pi^{T}_{\varepsilon} && \Delta(\beta) 
\end{pmatrix}
\ , \ $ where $\pi\mathbb{1}_{m} = \alpha,\ \pi^{T}\mathbb{1}_{n}=\beta,\ $ and $\Delta = diag: \mathbb{R}^{n} \rightarrow M_{n}(\mathbb{R})$ is the linear operator mapping a vector  to a diagonal matrix  containing  this vector.


This implies 
$$
\begin{pmatrix}
\Delta(\alpha) && \pi_{\varepsilon}\\
\pi^{T}_{\varepsilon} && \Delta(\beta) 
\end{pmatrix}
\begin{pmatrix}
\mathbb{1}_{n}\\
\mathbb{1}_{m}
\end{pmatrix} = 0\ ,
$$
that is,
$$
\begin{pmatrix}
\mathbb{1}_{n}\\
\mathbb{1}_{m}
\end{pmatrix}\in \ker(\nabla^{2}Q_{\alpha, \beta,\varepsilon}(f,g))\ .
$$
Hence, $\nabla^{2}Q_{\alpha, \beta,\varepsilon}(f,g)$ is singular. Therefore, on regularization we have the following Hessian
$
H_{reg} := \nabla^{2}Q_{\alpha, \beta,\varepsilon}(f,g)+\lambda cc^{T}\ ,
$ 
where $c= \begin{pmatrix}\frac{\mathbb{1}}{\sqrt{n+m}}\\-\frac{\mathbb{1}}{\sqrt{n+m}}\end{pmatrix}\in M_{(n+m),1}(\mathbb{R})$.

Now, at the $k^{th}$ iteration solve
$\nabla^{2}Q_{\alpha, \beta,\varepsilon}(f,g)p_{k} = \nabla Q_{\alpha, \beta,\varepsilon}(f,g)$ to obtain the optimizing direction vector $p_{k}$ and then perform the Armijo condition to obtain the update step $\alpha_{k}$ such that we have the update
$$
(f,g) \leftarrow (f,g) + \alpha_{k} p_{k}\ .
$$


In [ ]:
# Damped Newton
print("Damped Newton... ")
print( "Doing for (",N[0], N[1],")." )
rho = 0.95
c = 0.05
dampedNewtonP = []
results_dampedNewton  = []
times_dampedNewton    = []
Hessians_dampedNewton = []
#Cost matrix
C = distmat( x, y )
# a and b
a = normalize( np.ones( N[0] ) )
b = normalize( np.ones( N[1] ) )
epsilons = [ 1.0, 0.5, 0.1, 0.05, 0.03 ]
for eps in epsilons:
    print( "For epsilon = "+str(eps)+":" )    
    #Kernel
    K = np.exp( - C/eps )
    f, g = a, b
    print( " |- Iterating" )  
    start = time.time()
    Optimizer = computational_OT.damped_Newton(     K,
                                                    a,
                                                    b,
                                                    f,
                                                    g,
                                                    eps,
                                                    rho,
                                                    c )
    out = Optimizer._update(    max_iterations = 50,
                                debug = False )
    end = time.time()
    if out != -1:
        results_dampedNewton.append( out )
        times_dampedNewton.append( end - start )
        print( " |- Computing P" )
        print( "" )
        u_opt = np.exp( out['potential_f']/eps )
        K = np.exp( - C/eps )
        v_opt =  np.exp( out['potential_g']/eps )
        P_opt = GetP( u_opt, K, v_opt )
        dampedNewtonP.append( P_opt )
        print( " |- Recording (unstabilized) Hessian \n" )
        mat  = - eps * Optimizer.Hessian
        diag = 1/np.sqrt( np.concatenate( ( a, b ), axis = None ) )
        mat = diag[:,None] * mat * diag[None,:]
        Hessians_dampedNewton.append( mat )
    else:
        epsilons.remove( eps )
# end for

### Error plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )
for i in range(len(results_dampedNewton)):
  error = np.asarray( results_dampedNewton[i]['error_a'] ) + np.asarray( results_dampedNewton[i]['error_b'] )
  plt.plot( error, label = 'Damped Newton for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Error in log-scale" )
plt.legend( loc = "upper right" )
plt.yscale( 'log' )
plt.savefig( "../Images/DampedNewtonPreconditioningAllversions_images/ErrorDampedNewtonwithoutPrecond.pdf", format = 'pdf' )
plt.show()

print( "\n Error plots can increase! The error is not the objective function!" )

### Objective function plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.title( "$$" )
plt.title( "Objective Function" )
for i in range(len(results_dampedNewton)):
  plt.plot( np.asarray( results_dampedNewton[i]["objective_values"] ), label = 'Damped Newton for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Objective value" )
plt.legend( loc = "upper right" )
plt.savefig( "../Images/DampedNewtonPreconditioningAllversions_images/ObjectiveDampedNewtonwithoutPrecond.pdf", format = 'pdf' )
plt.show()


### Ascent step-size plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.subplot( 2, 1, 1 ),
plt.title( "Alpha" )
for i in range(len(results_dampedNewton)):
  plt.plot( np.asarray( results_dampedNewton[i]['linesearch_steps'] ), label = 'Damped Newton for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Alpha in log-scale" )
plt.legend()
plt.savefig( "../Images/DampedNewtonPreconditioningAllversions_images/AlphaDampedNewtonwithoutPrecond.pdf", format = 'pdf' )
plt.show()

# I. Plotting spectrum as a function of $\varepsilon$

In [ ]:
def spectral_decomposition( mat ):
    eig, v = np.linalg.eigh( mat )
    sorting_indices = np.argsort( eig )
    eig = eig[ sorting_indices ]
    v   = v[ : , sorting_indices ]
    print( "List of smallest eigenvalues: ", eig[ : 10 ] )
    print( "List of largest  eigenvalues: ", eig[ - 10 : ] )
    return eig, v

In [ ]:
eigs = []
eigvecs = []
for i in range( len(epsilons) ) :
    eps = epsilons[i]
    print( "Spectral statistics of Hessian for epsilon = "+str(eps) )
    ev = spectral_decomposition( Hessians_dampedNewton[i] )
    eigs.append( ev[0] )
    eigvecs.append( ev[1] )
    print("")
# end for

In [ ]:
fig, ax = plt.subplots( figsize = ( 5, 12 ), nrows = len(epsilons), ncols = 1, sharey = True )
plt.title( "Histogram of eigenvalues." )
for i in range( len(epsilons) ):
    ax[i].hist( eigs[i], 50 )
    ax[i].set_title( " $\epsilon$: "+str(epsilons[i]) )
    ax[i].set_xlabel( "Eigenvalues" )
    ax[i].set_yscale( "log" )
# end for
plt.subplots_adjust( wspace = 0, hspace = 0.5 )
plt.savefig( "../Images/DampedNewtonPreconditioningAllversions_images/eigenhistunstabilizedDampedNewtonwithoutPrecond.pdf", format = 'pdf' )
plt.show()

## Damped Newton with Preconditioning
Here we perform dual damped Newton with preconditioning. Here we consider $t$ eigenvalues of the Hessian that we want to move to one and form the following preconditioning matrix using the corresponding eigenvectors,
$$
P = \left(I_{n+m}-\sum_{i-1}^{t}\left(1 - \frac{1}{\sqrt{\lambda_{i}}}\right)y_{i}y_{i}^{T}\right)\ ,
$$
where
$$
y_{i} \in \ker\left(\nabla^{2}_{f}Q_{\alpha, \beta, \varepsilon}(f)-\lambda_{i}I_{n}\right),\ \forall i= 1,\dots,k\ ,
$$
 are orthonormal.

Now, at the $k^{th}$ iteration we solve the following equation:
$$
(P\nabla^{2}Q_{\alpha, \beta, \varepsilon}(f)P)(Pp_{k})=P\nabla Q_{\alpha, \beta, \varepsilon}(f)\ ,
$$
using iterative inversion methods such as "Conjugate gradient" and "GMRES" to get the ascent direction $p_{k}$, following which we use the Armijo condition to obtain the ascent step size $\alpha_{k}$.

# II. Actual preconditioning

In [ ]:
def build_preconditioners( num_eigs, modified_Hessian, ansatz = True ):
    # Diagonalize
    eigenvalues, eigenvectors = np.linalg.eigh( modified_Hessian )
    sorting_indices = np.argsort( eigenvalues )
    eigenvalues  = eigenvalues[ sorting_indices ]
    eigenvectors = eigenvectors[ : , sorting_indices ]
    # Form null vector
    if not ansatz:
        null_vector = eigenvectors[ : , 0 ]
    else:
        null_vector = np.hstack( ( np.ones(N[0]), - np.ones(N[1]) ) )
        norm = np.sqrt( N[0] + N[1] )
        null_vector = null_vector/norm
    # Form other vectors (only 13)
    _,m = eigenvectors.shape
    indices = []
    for i in range(num_eigs//2):
        indices.append( m -i - 2 )
        indices.append( i + 1 )
    # end for
    if num_eigs%2 != 0:
        indices.append( m - 1 - ( num_eigs//2 ) )
   
    precond_vectors = eigenvectors[ : , indices ]
    precond_vectors = []
    for index in indices:
        precond_vectors.append( eigenvectors[:,index] )
    # end for
    return null_vector, precond_vectors

In [ ]:
num_eigs = 13
null_vector, precond_vectors = build_preconditioners( num_eigs, Hessians_dampedNewton[-1], ansatz = False )

## Version 1

In [ ]:
# Damped Newton with preconditioning
print(" Damped Newton with preconditioning... ")
print( "Doing for (",N[0], N[1],")." )
rho = 0.95
c = 0.05
reset_starting_point = True
final_modified_Hessians_V1 = []
dampedNewtonwithprecondP_V1 = []
results_dampedNewtonwithprecond_V1  = []
times_dampedNewtonwithprecond_V1    = []
# Cost matrix
C = distmat( x, y )
# a and b
a = normalize( np.ones(N[0]) )
b = normalize( np.ones(N[1]) )
precond_epsilons = [ 1.0, 0.5, 0.1, 0.05 ]
f, g = None, None
for eps in precond_epsilons:
    print( "For epsilon = "+str(eps)+":" )  
    #Kernel
    K = np.exp( - C/eps )
    if (f is None) or (g is None): 
        f, g = a, b
    print( " |- Iterating" )   
    start = time.time()
    Optimizer = computational_OT.damped_Newton_with_preconditioning(    K,
                                                                        a,
                                                                        b,
                                                                        f,
                                                                        g,
                                                                        eps,
                                                                        rho,
                                                                        c,
                                                                        null_vector,
                                                                        precond_vectors[:] )
    out = Optimizer._update(    max_iterations = 50,
                                iterative_inversion = 30,
                                version = 1,
                                debug = False )
    results_dampedNewtonwithprecond_V1.append( out )
    end = time.time()
    times_dampedNewtonwithprecond_V1.append( end - start )
    print( " |- Computing P" )
    print( "" )
    u_opt = np.exp( out['potential_f']/eps )
    K = np.exp( - C/eps )
    v_opt =  np.exp( out['potential_g']/eps )
    P_opt = GetP( u_opt, K, v_opt )
    dampedNewtonwithprecondP_V1.append( P_opt )
    if not reset_starting_point:
        f = Optimizer.x[:a.shape[0]]
        g = Optimizer.x[a.shape[0]:]
    final_modified_Hessians_V1.append( Optimizer.modified_Hessian )
# end for

#### Averaged Time plot

In [ ]:
text = [ "Initial preconditioning",
        "Preconditioning matrix formation",
        "Debug Step",
        "Invert the linear system for p_k",
        "Unwinding",
        " Complete code block" 
       ]
plt.figure( figsize = ( 20, 10 ) )  
for j in range( len(results_dampedNewtonwithprecond_V1[0]['timings'][0]) ):
  values = []
  for i in range( len(results_dampedNewtonwithprecond_V1) ):
    mean = 0
    for k in range( len(results_dampedNewtonwithprecond_V1[i]['timings']) ):
      mean += results_dampedNewtonwithprecond_V1[i]['timings'][k][j]
    # end for
    mean = mean/len( results_dampedNewtonwithprecond_V1[i]['timings'] ) 
    values.append( mean )
  # end for
  if len(precond_epsilons) == len(values):
    plt.plot( precond_epsilons, np.asarray(values), label = text[j], linewidth = 2 )
    plt.legend( loc = 'upper right' )
# end for
plt.xlabel( "$\epsilon$" )
plt.ylabel( "Time in ms" )
plt.show()

#### Time plot

In [ ]:
text = [ "Initial preconditioning",
       "Preconditioning matrix formation",
       "Debug Step",
        "Invert the linear system for p_k",
        "Unwinding",
        "Complete code block" 
       ]
for i in range( len(results_dampedNewtonwithprecond_V1) ):
  plt.figure( figsize = ( 20, 10 ) ) 
  plt.title( "Epsilon: "+str(epsilons[i]) ) 
  for j in range( len(results_dampedNewtonwithprecond_V1[0]['timings'][0]) ):
    values = []
    for k in range( len(results_dampedNewtonwithprecond_V1[i]['timings']) ):
      values.append( results_dampedNewtonwithprecond_V1[i]['timings'][k][j] )
    # end for
    plt.plot( np.asarray(values), label = text[j], linewidth = 2 )
  # end for
  plt.legend( loc = 'upper right' )
  plt.xlabel( "Iterations" )
  plt.ylabel( "Time in ms" )
  plt.show()
# end for

#### Error plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" ) 
for i in range( len(results_dampedNewtonwithprecond_V1) ):
  error = np.asarray( results_dampedNewtonwithprecond_V1[i]['error_a'] ) + np.asarray( results_dampedNewtonwithprecond_V1[i]['error_b'] )
  plt.plot( error, label = 'Damped Newton with preconditioning for $\epsilon = $'+ str(precond_epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Error in log-scale" )
plt.legend( loc = "upper right" )
plt.yscale( 'log' )
plt.savefig( "../Images/DampedNewtonPreconditioningAllversions_images/ErrorDampedNewtonwithoutPrecondV1.pdf", format = 'pdf' )
plt.show()

print( "\n Error plots can increase! The error is not the objective function!" )

#### Objective function plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.title( "$$" )
plt.title( "Objective Function" )
for i in range( len(results_dampedNewtonwithprecond_V1) ):
  plt.plot( np.asarray(results_dampedNewtonwithprecond_V1[i]['objective_values']), label = 'Damped Newton with preconditioning for $\epsilon = $'+ str(precond_epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Objective value" )
plt.yscale( 'log' )
plt.legend( loc = "upper right" )
plt.savefig( "../Images/DampedNewtonPreconditioningAllversions_images/ObjectiveDampedNewtonwithPrecondV1.pdf", format = 'pdf')
plt.show()

#### Ascent step-size plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.subplot( 2, 1, 1 ),
plt.title( "Alpha" )
for i in range( len(results_dampedNewtonwithprecond_V1) ):
  plt.plot( np.asarray(results_dampedNewtonwithprecond_V1[i]['linesearch_steps']), label = 'Damped Newton with preconditioning for $\epsilon = $'+ str(precond_epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Alpha in log-scale" )
plt.legend( loc = "upper right" )
plt.savefig( "../Images/DampedNewtonPreconditioningAllversions_images/AlphaDampedNewtonwithPrecondV1.png", format = 'pdf' )
plt.show()


##  Version 2

In [ ]:
# Damped Newton with preconditioning
print(" Damped Newton with preconditioning... ")
print( "Doing for (",N[0], N[1],")." )
rho = 0.95
c = 0.05
reset_starting_point = True
final_modified_Hessians_V2 = []
dampedNewtonwithprecondP_V2 = []
results_dampedNewtonwithprecond_V2  = []
times_dampedNewtonwithprecond_V2    = []
# Cost matrix
C = distmat( x, y )
# a and b
a = normalize( np.ones( N[0] ) )
b = normalize( np.ones( N[1] ) )
precond_epsilons = [ 1.0, 0.5, 0.1, 0.05 ]
f, g = None, None
for eps in precond_epsilons:
    print( "For epsilon = "+str(eps)+":" )  
    #Kernel
    K = np.exp( -C/eps )
    if (f is None) or (g is None): 
        f, g = a, b
    print( " |- Iterating" )  
    start = time.time()
    Optimizer = computational_OT.damped_Newton_with_preconditioning(    K,
                                                                        a,
                                                                        b,
                                                                        f,
                                                                        g,
                                                                        eps,
                                                                        rho,
                                                                        c,
                                                                        null_vector,
                                                                        precond_vectors[:] )
    out = Optimizer._update(    max_iterations = 50,
                                iterative_inversion = 30,
                                version = 2,
                                debug = False )
    results_dampedNewtonwithprecond_V2 .append( out )
    end = time.time()
    times_dampedNewtonwithprecond_V2.append( end - start )
    print( " |- Computing P" )
    print( "" )
    u_opt = np.exp( out['potential_f']/eps )
    K = np.exp( - C/eps )
    v_opt =  np.exp( out['potential_g']/eps )
    P_opt = GetP( u_opt, K, v_opt )
    dampedNewtonwithprecondP_V2.append( P_opt )
    if not reset_starting_point:
        f = Optimizer.x[:a.shape[0]]
        g = Optimizer.x[a.shape[0]:]
    final_modified_Hessians_V2.append( Optimizer.modified_Hessian )
# end for

#### Averaged Time plot

In [ ]:
text = [
        "Initial preconditioning",
        "Preconditioning matrix formation",
        "Changing A to PAP",
        "Debug step",
        "Invert the linear system for p_k",
        "Unwinding",
        " Complete code block"
        ]
plt.figure( figsize = ( 20, 10 ) )  
for j in range( len(results_dampedNewtonwithprecond_V1[0]['timings'][0]) ):
  values = []
  for i in range( len(results_dampedNewtonwithprecond_V1) ):
    mean = 0
    for k in range( len(results_dampedNewtonwithprecond_V2[i]['timings']) ):
      mean += results_dampedNewtonwithprecond_V2[i]['timings'][k][j]
    # end for
    mean = mean/len( results_dampedNewtonwithprecond_V2[i]['timings'] ) 
    values.append( mean )
  # end for
  if len(precond_epsilons) == len(values):
    plt.plot( precond_epsilons, np.asarray(values), label = text[j], linewidth = 2 )
    plt.legend( loc = "upper right" )
# end for
plt.xlabel( "$\epsilon$" )
plt.ylabel( "Time in ms" )
plt.show()

#### Time plot

In [ ]:
text = [
        "Initial preconditioning",
        "Preconditioning matrix formation",
        "Changing A to PAP",
        "Debug step",
        "Invert the linear system for p_k",
        "Unwinding",
        " Complete code block"
        ]
for i in range(len(results_dampedNewtonwithprecond_V2)):
  plt.figure( figsize = ( 20, 10 ) ) 
  plt.title( "Epsilon: "+str(epsilons[i]) ) 
  for j in range( len(results_dampedNewtonwithprecond_V2[0]['timings'][0]) ):
    values = []
    for k in range( len(results_dampedNewtonwithprecond_V2[i]['timings']) ):
      values.append( results_dampedNewtonwithprecond_V2[i]['timings'][k][j] )
    # end for
    plt.plot( np.asarray(values),label = text[j],linewidth = 2 )
  # end for
  plt.legend( loc = 'upper right' )
  plt.xlabel( "Iterations" )
  plt.ylabel( "Time in ms" )
  plt.show()
# end for

#### Error plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )
for i in range( len(results_dampedNewtonwithprecond_V2) ):
  error = np.asarray( results_dampedNewtonwithprecond_V2[i]['error_a'] ) + np.asarray( results_dampedNewtonwithprecond_V2[i]['error_b'] )
  plt.plot( error, label = 'Damped Newton with preconditioning for $\epsilon = $'+ str(precond_epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Error in log-scale" )
plt.legend( loc = "upper right" )
plt.yscale( 'log' )
plt.savefig( "../Images/DampedNewtonPreconditioningAllversions_images/ErrorDampedNewtonwithPrecondV2.pdf", format = 'pdf' )
plt.show()

print( "\n Error plots can increase! The error is not the objective function!" )

#### Objective function plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.title( "$$" )
plt.title( "Objective Function" )
for i in range( len(results_dampedNewtonwithprecond_V2) ):
  plt.plot( np.asarray( results_dampedNewtonwithprecond_V2[i]['objective_values'] ), label = 'Damped Newton with preconditioning for $\epsilon = $'+ str(precond_epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Objective value" )
plt.yscale( 'log' )
plt.legend( loc = "upper right" )
plt.savefig( "../Images/DampedNewtonPreconditioningAllversions_images/ObjectiveDampedNewtonwithPrecondV2.pdf", format = 'pdf' )
plt.show()

#### Ascent step-size plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.subplot( 2, 1, 1 ),
plt.title( "Alpha" )
for i in range(len(results_dampedNewtonwithprecond_V2)):
  plt.plot( np.asarray(results_dampedNewtonwithprecond_V2[i]['linesearch_steps']), label = 'Damped Newton with preconditioning for $\epsilon = $'+ str(precond_epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Alpha in log-scale" )
plt.legend( loc = "upper right" )
plt.savefig( "../Images/DampedNewtonPreconditioningAllversions_images/AlphaDampedNewtonwithPrecondV2.pdf", format = 'pdf' )
plt.show()

## Version 3

#### Conjugate Gradient

In [ ]:
# Damped Newton with preconditioning
print(" Damped Newton with preconditioning... ")
print( "Doing for (",N[0], N[1],")." )
rho = 0.95
c = 0.05
reset_starting_point = True
final_modified_Hessians_V3_cg = []
dampedNewtonwithprecondP_V3_cg = []
results_dampedNewtonwithprecond_V3_cg  = []
times_dampedNewtonwithprecond_V3_cg    = []
# Cost matrix
C = distmat( x, y )
# a and b
a = normalize( np.ones( N[0] ) )
b = normalize( np.ones( N[1] ) )
precond_epsilons = [ 1.0, 0.5, 0.1, 0.05 ]
f, g = None, None
for eps in precond_epsilons:
    print( "For epsilon = "+str(eps)+":" )  
    #Kernel
    K = np.exp( - C/eps )
    if (f is None) or (g is None): 
        f, g = a, b
    print( " |- Iterating" )  
    start = time.time()
    Optimizer = computational_OT.damped_Newton_with_preconditioning(    K,
                                                                        a,
                                                                        b,
                                                                        f,
                                                                        g,
                                                                        eps,
                                                                        rho,
                                                                        c,
                                                                        null_vector,
                                                                        precond_vectors[:] )
    out = Optimizer._update(    max_iterations = 50,
                                iterative_inversion = 30,
                                version = 3,
                                debug = False,
                                optType = 'cg' )
    results_dampedNewtonwithprecond_V3_cg.append( out )
    end = time.time()
    times_dampedNewtonwithprecond_V3_cg.append( end - start )
    print( " |- Computing P" )
    print( "" )
    u_opt = np.exp( out['potential_f']/eps )
    K = np.exp( - C/eps )
    v_opt =  np.exp( out['potential_g']/eps )
    P_opt = GetP( u_opt, K, v_opt )
    dampedNewtonwithprecondP_V3_cg.append( P_opt )
    if not reset_starting_point:
        f = Optimizer.x[:a.shape[0]]
        g = Optimizer.x[a.shape[0]:]
    final_modified_Hessians_V3_cg.append( Optimizer.modified_Hessian )
# end for

##### Averaged Time plots

In [ ]:
text = [
        "Initial Preconditioning",
        " Preconditioning matrix formation",
        "Changing A to PAP",
        "Invert the linear system for p_k",
        "Unwinding",
        "Complete code block"
        ]
plt.figure( figsize = ( 20, 10 ) )  
for j in range( len(results_dampedNewtonwithprecond_V3_cg[0]['timings'][0]) ):
  values = []
  for i in range( len(results_dampedNewtonwithprecond_V3_cg) ):
    mean = 0
    for k in range( len(results_dampedNewtonwithprecond_V3_cg[i]['timings']) ):
      mean += results_dampedNewtonwithprecond_V3_cg[i]['timings'][k][j]
    # end for
    mean = mean/len(results_dampedNewtonwithprecond_V3_cg[i]['timings']) 
    values.append( mean )
  # end for
  if len(precond_epsilons) == len(values):
    plt.plot( precond_epsilons, np.asarray(values), label = text[j], linewidth = 2 )
    plt.legend( loc = "upper right" )
# end for
plt.xlabel( "$\epsilon$" )
plt.ylabel( "Time in ms" )
plt.show()

##### Time plot

In [ ]:
text = [
        "Initial Preconditioning",
        " Preconditioning matrix formation",
        "Changing A to PAP",
        " Invert the linear system for p_k",
        "Unwinding",
        " complete code block"
        ]
for i in range(len(results_dampedNewtonwithprecond_V3_cg)):
  plt.figure( figsize = ( 20, 10 ) ) 
  plt.title("Epsilon: "+str(epsilons[i])) 
  for j in range(len(results_dampedNewtonwithprecond_V3_cg[0]['timings'][0])):
    values = []
    for k in range(len(results_dampedNewtonwithprecond_V3_cg[i]['timings'])):
      values.append( results_dampedNewtonwithprecond_V3_cg[i]['timings'][k][j] )
    # end for
    plt.plot(np.asarray(values), label = text[j], linewidth = 2)
  # end for
  plt.legend( loc = 'upper right' )
  plt.xlabel( "Iterations" )
  plt.ylabel( "Time in ms" )
  plt.show()
# end for

##### Error plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )
for i in range( len(results_dampedNewtonwithprecond_V3_cg) ):
  error = np.asarray( results_dampedNewtonwithprecond_V3_cg[i]['error_a'] ) + np.asarray( results_dampedNewtonwithprecond_V3_cg[i]['error_b'] )
  plt.plot( error, label = 'Damped Newton with preconditioning for $\epsilon = $' + str(precond_epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Error in log-scale" )
plt.legend( loc = "upper right" )
plt.yscale( 'log' )
plt.savefig( "../Images/DampedNewtonPreconditioningAllversions_images/ErrorDampedNewtonwithPrecondV3_cg.pdf", format = 'pdf' )
plt.show()
print( "\n Error plots can increase! The error is not the objective function!" )

##### Objective function plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.title( "$$" ) 
plt.title( "MObjective Function" )
for i in range( len(results_dampedNewtonwithprecond_V3_cg) ):
  plt.plot( np.asarray( results_dampedNewtonwithprecond_V3_cg[i]['objective_values'] ), label = 'Damped Newton with preconditioning for $\epsilon = $'+ str(precond_epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Objective value" )
plt.yscale( 'log' )
plt.legend( loc = "upper right" )
plt.savefig( "../Images/DampedNewtonPreconditioningAllversions_images/ObjectiveDampedNewtonwithPrecondV3_cg.pdf", format = 'pdf' )
plt.show()

##### Ascent step-size plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.subplot( 2, 1, 1 ),
plt.title( "Alpha" )
for i in range( len(results_dampedNewtonwithprecond_V3_cg) ):
  plt.plot( np.asarray(results_dampedNewtonwithprecond_V3_cg[i]['linesearch_steps']), label = 'Damped Newton with preconditioning for $\epsilon = $'+ str(precond_epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Alpha in log-scale" )
plt.legend( loc = "upper right" )
plt.savefig( "../Images/DampedNewtonPreconditioningAllversions_images/AlphaDampedNewtonwithPrecond3_cg.pdf", format = 'pdf' )
plt.show()


#### GMRES

In [ ]:
# Damped Newton with preconditioning
print(" Damped Newton with preconditioning... ")
print( "Doing for (",N[0], N[1],")." )
rho = 0.95
c = 0.05
reset_starting_point = True
final_modified_Hessians_V3_gmres = []
dampedNewtonwithprecondP_V3_gmres = []
results_dampedNewtonwithprecond_V3_gmres  = []
times_dampedNewtonwithprecond_V3_gmres    = []
# Cost matrix
C = distmat( x, y )
# a and b
a = normalize( np.ones( N[0] ) )
b = normalize( np.ones( N[1] ) )
precond_epsilons = [ 1.0, 0.5, 0.1, 0.05 ]
f, g = None, None
for eps in precond_epsilons:
    print( "For epsilon = "+str(eps)+":" )  
    #Kernel
    K = np.exp( - C/eps )
    if (f is None) or (g is None): 
        f, g = a, b
    print( " |- Iterating" )  
    start = time.time()
    Optimizer = computational_OT.damped_Newton_with_preconditioning(    K,
                                                                        a,
                                                                        b,
                                                                        f,
                                                                        g,
                                                                        eps,
                                                                        rho,
                                                                        c,
                                                                        null_vector,
                                                                        precond_vectors[:] )
    out = Optimizer._update(    max_iterations = 50,
                                iterative_inversion = 30,
                                version = 3,
                                debug = False,
                                optType = 'gmres' )
    results_dampedNewtonwithprecond_V3_gmres.append( out )
    end = time.time()
    times_dampedNewtonwithprecond_V3_gmres.append( end - start )
    print( " |- Computing P" )
    print( "" )
    u_opt = np.exp( out['potential_f']/eps )
    K = np.exp( - C/eps )
    v_opt =  np.exp( out['potential_g']/eps )
    P_opt = GetP( u_opt, K, v_opt )
    dampedNewtonwithprecondP_V3_gmres.append( P_opt )
    if not reset_starting_point:
        f = Optimizer.x[:a.shape[0]]
        g = Optimizer.x[a.shape[0]:]
    final_modified_Hessians_V3_gmres.append( Optimizer.modified_Hessian )
# end for

##### Averaged Time plot

In [ ]:
text = [
        "Initial Preconditioning",
        " Preconditioning matrix formation",
        "Changing A to PAP",
        " Invert the linear system for p_k",
        "Unwinding",
        " complete code block"
]
plt.figure( figsize = ( 20, 10 ) )  
for j in range( len(results_dampedNewtonwithprecond_V3_gmres[0]['timings'][0]) ):
  values = []
  for i in range( len(results_dampedNewtonwithprecond_V3_gmres) ):
    mean = 0
    for k in range( len(results_dampedNewtonwithprecond_V3_gmres[i]['timings']) ):
      mean += results_dampedNewtonwithprecond_V3_gmres[i]['timings'][k][j]
    # end for
    mean = mean/len( results_dampedNewtonwithprecond_V3_gmres[i]['timings'] ) 
    values.append( mean )
  # end for
  if len(precond_epsilons) == len(values):
    plt.plot( precond_epsilons, np.asarray(values), label = text[j], linewidth = 2 )
    plt.legend( loc = "upper right" )
# end for
plt.xlabel( "$\epsilon$" )
plt.ylabel( "Time in ms" )
plt.show()

##### Time plot

In [ ]:
text = [
        "Initial Preconditioning",
        " Preconditioning matrix formation",
        "Changing A to PAP",
        " Invert the linear system for p_k",
        "Unwinding",
        " complete code block"
        ]
for i in range( len(results_dampedNewtonwithprecond_V3_gmres) ):
  plt.figure( figsize = ( 20, 10 ) ) 
  plt.title( "Epsilon: "+str(epsilons[i]) ) 
  for j in range( len(results_dampedNewtonwithprecond_V3_gmres[0]['timings'][0]) ):
    values = []
    for k in range( len(results_dampedNewtonwithprecond_V3_gmres[i]['timings']) ):
      values.append( results_dampedNewtonwithprecond_V3_gmres[i]['timings'][k][j] )
    # end for
    plt.plot( np.asarray(values), label = text[j], linewidth = 2 )
  # end for
  plt.legend( loc = 'upper right' )
  plt.xlabel( "Iterations" )
  plt.ylabel( "Time in ms" )
  plt.show()
# end for

##### Error plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )
for i in range( len(results_dampedNewtonwithprecond_V3_gmres) ):
  error = np.asarray( results_dampedNewtonwithprecond_V3_gmres[i]['error_a'] ) + np.asarray( results_dampedNewtonwithprecond_V3_gmres[i]['error_b'] )
  plt.plot( error, label = 'Damped Newton with preconditioning for $\epsilon = $'+ str(precond_epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Error in log-scale" )
plt.legend( loc = "upper right" )
plt.yscale( 'log' )
plt.savefig( "../Images/DampedNewtonPreconditioningAllversions_images/ErrorDampedNewtonwithPrecondV3_gmres.pdf", format = 'pdf' )
plt.show()
print( "\n Error plots can increase! The error is not the objective function!" )

##### Objective function plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.title( "$$" )
plt.title( "Objective Function" )
for i in range( len(results_dampedNewtonwithprecond_V3_gmres) ):
  plt.plot( -np.asarray(results_dampedNewtonwithprecond_V3_gmres[i]['objective_values']), label = 'Damped Newton with preconditioning for $\epsilon = $'+ str(precond_epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Objective value" )
plt.yscale( 'log' )
plt.legend( loc = "upper right" )
plt.savefig( "../Images/DampedNewtonPreconditioningAllversions_images/ObjectiveDampedNewtonwithPrecondV3_gmres.pdf", format = 'pdf' )
plt.show()

##### Ascent step-size plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.subplot( 2, 1, 1 ),
plt.title( "Alpha" )
for i in range( len(results_dampedNewtonwithprecond_V3_gmres) ):
  plt.plot( np.asarray(results_dampedNewtonwithprecond_V3_gmres[i]['linesearch_steps']), label = 'Damped Newton with preconditioning for $\epsilon = $'+ str(precond_epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Alpha in log-scale" )
plt.legend( loc = "upper right" )
plt.savefig( "../Images/DampedNewtonPreconditioningAllversions_images/AlphaDampedNewtonwithPrecondV3_gmres.pdf", format = 'pdf' )
plt.show()


## Version 4

In [ ]:
# Damped Newton with preconditioning
print(" Damped Newton with preconditioning... ")
print( "Doing for (",N[0], N[1],")." )
rho = 0.95
c = 0.05
reset_starting_point = True
final_modified_Hessians_V4 = []
dampedNewtonwithprecondP_V4 = []
results_dampedNewtonwithprecond_V4  = []
times_dampedNewtonwithprecond_V4    = []
# Cost matrix
C = distmat( x, y )
# a and b
a = normalize( np.ones( N[0] ) )
b = normalize( np.ones( N[1] ) )
precond_epsilons = [ 1.0, 0.5, 0.1, 0.05 ]
f, g = None, None
for eps in precond_epsilons:
    print( "For epsilon = "+str(eps)+":" )  
    #Kernel
    K = np.exp( - C/eps )
    if (f is None) or (g is None): 
        f, g = a, b
    print( " |- Iterating" )  
    start = time.time()
    Optimizer = computational_OT.damped_Newton_with_preconditioning(    K,
                                                                        a,
                                                                        b,
                                                                        f,
                                                                        g,
                                                                        eps,
                                                                        rho,
                                                                        c,
                                                                        null_vector,
                                                                        precond_vectors[:] )
    out = Optimizer._update(    max_iterations = 50,
                                iterative_inversion = 30,
                                version = 4,
                                debug = False,
                                optType = 'cg' )
    results_dampedNewtonwithprecond_V4.append( out )
    end = time.time()
    times_dampedNewtonwithprecond_V4.append( end - start )
    print( " |- Computing P" )
    print( "" )
    u_opt = np.exp( out['potential_f']/eps )
    K = np.exp( - C/eps )
    v_opt =  np.exp( out['potential_g']/eps )
    P_opt = GetP( u_opt, K, v_opt )
    dampedNewtonwithprecondP_V4.append( P_opt )
    if not reset_starting_point:
        f = Optimizer.x[:a.shape[0]]
        g = Optimizer.x[a.shape[0]:]
    final_modified_Hessians_V4.append( Optimizer.modified_Hessian )
# end for

#### Error plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )
for i in range( len(results_dampedNewtonwithprecond_V4) ):
  error = np.asarray( results_dampedNewtonwithprecond_V4[i]['error_a'] ) + np.asarray( results_dampedNewtonwithprecond_V4[i]['error_b'] )
  plt.plot( error, label = 'Damped Newton with preconditioning for $\epsilon = $'+ str(precond_epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Error in log-scale" )
plt.legend( loc = "upper right" )
plt.yscale( 'log' )
plt.savefig( "../Images/DampedNewtonPreconditioningAllversions_images/ErrorDampedNewtonwithPrecondV4_cg.pdf", format = 'pdf' )
plt.show()
print( "\n Error plots can increase! The error is not the objective function!" )

#### Averaged Time plot

In [ ]:
text = [
        "Preconditioning 1: Form E data",
        "Preconditioning 2: Form P data",
        "Form preconditioning functions",
        "Invert the linear system for p_k",
        "Unwinding",
        "Complete code block"
        ]
plt.figure( figsize = ( 20, 10 ) )  
for j in range( len(results_dampedNewtonwithprecond_V4[0]['timings'][0]) ):
  values = []
  for i in range( len(results_dampedNewtonwithprecond_V4) ):
    mean = 0
    for k in range( len(results_dampedNewtonwithprecond_V4[i]['timings']) ):
      mean += results_dampedNewtonwithprecond_V4[i]['timings'][k][j]
    # end for
    mean = mean/len( results_dampedNewtonwithprecond_V4[i]['timings'] ) 
    values.append( mean )
  # end for
  if len(precond_epsilons) == len(values):
    plt.plot( precond_epsilons, np.asarray(values), label = text[j],linewidth = 2 )
    plt.legend( loc = "upper right" )
# end for
plt.xlabel( "$\epsilon$" )
plt.ylabel( "Time in ms" )
plt.show()

#### Time plot

In [ ]:
text = [
        "Initial Preconditioning",
        " Preconditioning matrix formation",
        "Changing A to PAP",
        " Invert the linear system for p_k",
        "Unwinding",
        " complete code block"
        ]
for i in range( len(results_dampedNewtonwithprecond_V4) ):
  plt.figure( figsize = ( 20, 10 ) ) 
  plt.title( "Epsilon: "+str(epsilons[i]) ) 
  for j in range( len(results_dampedNewtonwithprecond_V4[0]['timings'][0]) ):
    values = []
    for k in range( len(results_dampedNewtonwithprecond_V4[i]['timings']) ):
      values.append( results_dampedNewtonwithprecond_V4[i]['timings'][k][j] )
    # end for
    plt.plot( np.asarray(values), label = text[j], linewidth = 2 )
  # end for
  plt.legend( loc = "upper right" )
  plt.xlabel( "Iterations" )
  plt.ylabel( "Time in ms" )
  plt.show()
# end for

#### Objective function plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.title( "$$" )
plt.title( "Objective Function" )
for i in range( len(results_dampedNewtonwithprecond_V4) ):
  plt.plot( np.asarray(results_dampedNewtonwithprecond_V4[i]['objective_values']), label = 'Damped Newton with preconditioning for $\epsilon = $'+ str(precond_epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Objective value" )
plt.yscale( 'log' )
plt.legend( loc = "upper right" )
plt.savefig( "../Images/DampedNewtonPreconditioningAllversions_images/ObjectiveDampedNewtonwithPrecondV4.pdf", format = 'pdf' )
plt.show()

#### Ascent step-size plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.subplot( 2, 1, 1 ),
plt.title( "Alpha" )
for i in range( len(results_dampedNewtonwithprecond_V4) ):
  plt.plot( np.asarray(results_dampedNewtonwithprecond_V4[i]['linesearch_steps']), label = 'Damped Newton with preconditioning for $\epsilon = $'+ str(precond_epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Alpha in log-scale" )
plt.legend( loc = "upper right" )
plt.savefig( "../Images/DampedNewtonPreconditioningAllversions_images/AlphaDampedNewtonwithPrecondV4.pdf", format = 'pdf')
plt.show()

## Final version

In [ ]:
# Damped Newton with preconditioning
print(" Damped Newton with preconditioning... ")
print( "Doing for (",N[0], N[1],")." )
rho = 0.95
c = 0.05
reset_starting_point = True
final_modified_Hessians_V_final = []
dampedNewtonwithprecondP_V_final = []
results_dampedNewtoneithprecond_V_final  = []
times_dampedNewtonwithprecond_V_final    = []
# Cost matrix
C = distmat( x, y )
# a and b
a = normalize( np.ones( N[0] ) )
b = normalize( np.ones( N[1] ) )
precond_epsilons = [ 1.0, 0.5, 0.1, 0.05 ]
f, g = None, None
for eps in precond_epsilons:
    print( "For epsilon = "+str(eps)+":" )  
    #Kernel
    K = np.exp( - C/eps )
    if (f is None) or (g is None): 
        f, g = a, b
    print( " |- Iterating" )  
    start = time.time()
    Optimizer = computational_OT.damped_Newton_with_preconditioning(    K,
                                                                        a,
                                                                        b,
                                                                        f,
                                                                        g,
                                                                        eps,
                                                                        rho,
                                                                        c,
                                                                        null_vector,
                                                                        precond_vectors[:] )
    out = Optimizer._update(    max_iterations = 50,
                                iterative_inversion = 30,
                                version = None,
                                debug = False,
                                optType = 'cg' )
    results_dampedNewtoneithprecond_V_final.append( out )
    end = time.time()
    times_dampedNewtonwithprecond_V_final.append( end - start )
    print( " |- Computing P" )
    print( "" )
    u_opt = np.exp( out['potential_f']/eps )
    K = np.exp( - C/eps )
    v_opt =  np.exp( out['potential_g']/eps )
    P_opt = GetP( u_opt, K, v_opt )
    dampedNewtonwithprecondP_V_final.append( P_opt )
    if not reset_starting_point:
        f = Optimizer.x[:a.shape[0]]
        g = Optimizer.x[a.shape[0]:]
    final_modified_Hessians_V_final.append( Optimizer.modified_Hessian )
#end for

#### Error plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )
for i in range( len(results_dampedNewtoneithprecond_V_final) ):
  error = np.asarray( results_dampedNewtoneithprecond_V_final[i]['error_a'] ) + np.asarray( results_dampedNewtoneithprecond_V_final[i]['error_b'] )
  plt.plot( error, label = 'Damped Newton with preconditioning for $\epsilon = $'+ str(precond_epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Error in log-scale" )
plt.legend( loc = "upper right" )
plt.yscale( 'log' )
plt.savefig( "../Images/DampedNewtonPreconditioningAllversions_images/ErrorDampedNewtonwithPrecond_final_cg.pdf", format = 'pdf' )
plt.show()
print( "\n Error plots can increase! The error is not the objective function!" )

#### Averaged Time plot

In [ ]:
text = [
        "Preconditioning 1: Form E data",
        "Preconditioning 2: Form P data",
        "Form preconditioning functions",
        "Invert the linear system for p_k",
        "Unwinding",
        "Complete code block"
        ]
plt.figure( figsize = ( 20, 10 ) )  
for j in range( len(results_dampedNewtoneithprecond_V_final[0]['timings'][0]) ):
  values = []
  for i in range( len(results_dampedNewtoneithprecond_V_final) ):
    mean = 0
    for k in range( len(results_dampedNewtoneithprecond_V_final[i]['timings']) ):
      mean += results_dampedNewtoneithprecond_V_final[i]['timings'][k][j]
    # end for
    mean = mean/len( results_dampedNewtoneithprecond_V_final[i]['timings'] ) 
    values.append( mean )
  # end for
  if len(precond_epsilons) == len(values):
    plt.plot( precond_epsilons, np.asarray(values), label = text[j],linewidth = 2 )
    plt.legend( loc = "upper right" )
# end for
plt.xlabel( "$\epsilon$" )
plt.ylabel( "Time in ms" )
plt.savefig( "../Images/DampedNewtonPreconditioningAllversions_images/Timings_DampedNewtonwithPrecond_final_cg.pdf", format = 'pdf')
plt.show()

# III. More spectral statistics

In [ ]:
def print_spectral_statistics( mat, stabilize = False ):
    if stabilize:
        # Stabilizing largest and smallest eigenvalue
        min_vector = np.hstack( (np.ones(N[0]), - np.ones(N[1])) )
        max_vector = np.hstack( (np.ones(N[0]),  np.ones(N[1])) )
        norm = np.sqrt( N[0] + N[1] )
        min_vector = min_vector/norm
        max_vector = max_vector/norm
        min_vector = min_vector.reshape( (min_vector.shape[0], 1) )
        max_vector = max_vector.reshape( (max_vector.shape[0], 1) )
        #
        mat = mat + np.dot( min_vector, min_vector.T )
        mat = mat - np.dot( max_vector, max_vector.T )
    # endif
    eig, v = np.linalg.eigh( mat )
    sorting_indices = np.argsort( eig )
    eig = eig[ sorting_indices ]
    v   = v[ : , sorting_indices ]
    
    #print( "Mean eigenvalue: ", np.mean(eig) )
    print( "List of smallest eigenvalues: ", eig[ : 10 ] )
    print( "List of largest  eigenvalues: ", eig[ - 10 : ] )
    min_index = np.argmin(eig)
    max_index = np.argmax(eig)
    min_value = eig[ min_index ]
    max_value = eig[ max_index ]
    min_vector = v[ : , min_index ]
    min_vector = min_vector/min_vector[0]
    max_vector = v[:,max_index ]
    max_vector = max_vector/max_vector[0]
    condition_number = max_value/min_value
    # Test smallest and largest
    # print( "Min eigenvalue vector: ", min_vector)
    # print( "Max eigenvalue vector: ", max_vector)
    #
    #print( v[:,0]*np.sqrt( self.N1 + self.N2))
    #vector = v[:,0]
    #test = np.dot( result, vector)
    #print( np.linalg.norm(test) )
    #print("Min absolute eigenvalues: ", min_value)
    #print("Norm of v-1: ", np.linalg.norm(min_vector-eig_vector))
    print( "Condition number: ", condition_number )
    # plt.hist( eig, 50)
    # plt.title( "Histogram of eigenvalues for Hessian")
    # plt.xlabel( "Eigenvalues")
    # plt.yscale( "log" )
    # plt.show()
    return eig, v

In [ ]:
eigs = []
eigvecs = []
for i in range(len(epsilons)):
    eps = epsilons[i]
    print( "Spectral statistics of Hessian for epsilon = "+str(eps) )
    Hessian = Hessians_dampedNewton[i]
    ev = print_spectral_statistics( Hessian, stabilize = False )
    eigs.append(ev[0])
    eigvecs.append(ev[1])
    print("")
# end for


In [ ]:
# Using the previous epsilon's eigenvectors for conditioning
print( "Building preconditioning eigenvectors"  )
num_eigs = 13
null_vector, precond_vectors = build_preconditioners( num_eigs, Hessians_dampedNewton[-1], ansatz = False )
for i in range(len(precond_epsilons)):
    eps = precond_epsilons[i]
    print( "Spectral statistics of Hessian for epsilon = "+str(eps) )
    Hessian = final_modified_Hessians_V_final[i]
    print_spectral_statistics( Hessian)
    print( "" )
    print( "Building preconditioning eigenvectors" )
    num_eigs = 13
    null_vector, precond_vectors = build_preconditioners( num_eigs, Hessians_dampedNewton[-1], ansatz = False )
    print( "" )
# end for